In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [43]:
import torch
import numpy as np
from scipy.ndimage import gaussian_filter


def convert_rgb_to_ycbcr(img, dim_order='hwc'):
    if dim_order == 'hwc':
        y = 16. + (64.738 * img[..., 0] + 129.057 * img[..., 1] + 25.064 * img[..., 2]) / 256.
        cb = 128. + (-37.945 * img[..., 0] - 74.494 * img[..., 1] + 112.439 * img[..., 2]) / 256.
        cr = 128. + (112.439 * img[..., 0] - 94.154 * img[..., 1] - 18.285 * img[..., 2]) / 256.
    else:
        y = 16. + (64.738 * img[0] + 129.057 * img[1] + 25.064 * img[2]) / 256.
        cb = 128. + (-37.945 * img[0] - 74.494 * img[1] + 112.439 * img[2]) / 256.
        cr = 128. + (112.439 * img[0] - 94.154 * img[1] - 18.285 * img[2]) / 256.
    return np.array([y, cb, cr]).transpose([1, 2, 0])


def preprocess(img, device):
    img = np.array(img).astype(np.float32)
    ycbcr = convert_rgb_to_ycbcr(img)
    x = ycbcr[..., 0]
    x /= 255.
    x = torch.from_numpy(x).to(device)
    x = x.unsqueeze(0).unsqueeze(0)
    return x, ycbcr


def calc_psnr(img1, img2):
    return 10. * torch.log10(1. / torch.mean((img1 - img2) ** 2))


def calc_ssim(img1, img2, sd=1.5, C1=0.01**2, C2=0.03**2):
    mu1 = gaussian_filter(img1, sd)
    mu2 = gaussian_filter(img2, sd)
    mu1_sq = mu1 * mu1
    mu2_sq = mu2 * mu2
    mu1_mu2 = mu1 * mu2
    sigma1_sq = gaussian_filter(img1 * img1, sd) - mu1_sq
    sigma2_sq = gaussian_filter(img2 * img2, sd) - mu2_sq
    sigma12 = gaussian_filter(img1 * img2, sd) - mu1_mu2
    
    ssim_num = ((2 * mu1_mu2 + C1) * (2 * sigma12 + C2))
    ssim_den = ((mu1_sq + mu2_sq + C1) * (sigma1_sq + sigma2_sq + C2))
    ssim_map = ssim_num / ssim_den
    mssim = np.mean(ssim_map)
    
    return mssim


In [55]:
import os
import torch
import torch.backends.cudnn as cudnn
import numpy as np
import PIL.Image as pil_image


def test(image_file, scale, save=False, debug=False):
    image = pil_image.open(image_file).convert('RGB')
    image_file = os.path.basename(image_file)

    image_width = (image.width // scale) * scale
    image_height = (image.height // scale) * scale

    hr = image.resize((image_width, image_height), resample=pil_image.BICUBIC)
    lr = hr.resize((hr.width // scale, hr.height // scale), resample=pil_image.BICUBIC)
    bicubic = lr.resize((lr.width * scale, lr.height * scale), resample=pil_image.BICUBIC)

    if save:
        save_path = f'/content/drive/Shareddrives/BTP Meets/results/Set5/{scale}x/{image_file}'
        bicubic.save(save_path.replace('.', '_bicubic.'))

    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    lr, _ = preprocess(lr, device)
    hr, _ = preprocess(hr, device)
    bicubic, _ = preprocess(bicubic, device)

    psnr = calc_psnr(hr, bicubic)
    ssim = calc_ssim(hr, bicubic)
    if debug:
        print(f'PSNR/SSIM: {psnr:.2f}/{ssim:.4f}')

    return float(psnr), float(ssim)

In [56]:
import os


def do_test(psnr, ssim, BASE_DIR, save=False, debug=False):
    scales = [2, 3, 4, 8, 16]

    for file in os.listdir(BASE_DIR):
        if file.endswith(".png"):
            image_file_path = os.path.join(BASE_DIR, file)
            if debug:
                print(file)
            for scale in scales:
                if debug:
                    print(f"Scale: {scale}")
                result = test(image_file_path, scale, save, debug)
                if scale not in psnr:
                    psnr[scale] = []
                if scale not in ssim:
                    ssim[scale] = []
                psnr[scale].append(result[0])
                ssim[scale].append(result[1])
            if debug:
                print()

In [57]:
psnr = {}
ssim = {}
do_test(psnr, ssim, '/content/drive/Shareddrives/BTP Meets/datasets/test/Set5/', True, True)

head.png
Scale: 2
PSNR/SSIM: 34.95/0.8640
Scale: 3
PSNR/SSIM: 34.18/0.8478
Scale: 4
PSNR/SSIM: 31.74/0.7595
Scale: 8
PSNR/SSIM: 29.30/0.6815
Scale: 16
PSNR/SSIM: 27.01/0.6783

butterfly.png
Scale: 2
PSNR/SSIM: 27.50/0.9157
Scale: 3
PSNR/SSIM: 24.52/0.8318
Scale: 4
PSNR/SSIM: 22.18/0.7322
Scale: 8
PSNR/SSIM: 17.86/0.4919
Scale: 16
PSNR/SSIM: 15.18/0.3716

bird.png
Scale: 2
PSNR/SSIM: 36.77/0.9721
Scale: 3
PSNR/SSIM: 32.51/0.9240
Scale: 4
PSNR/SSIM: 30.08/0.8693
Scale: 8
PSNR/SSIM: 25.25/0.7011
Scale: 16
PSNR/SSIM: 21.74/0.5763

baby.png
Scale: 2
PSNR/SSIM: 37.15/0.9533
Scale: 3
PSNR/SSIM: 34.66/0.9185
Scale: 4
PSNR/SSIM: 31.88/0.8608
Scale: 8
PSNR/SSIM: 27.37/0.7396
Scale: 16
PSNR/SSIM: 24.44/0.6801

woman.png
Scale: 2
PSNR/SSIM: 32.20/0.9479
Scale: 3
PSNR/SSIM: 28.74/0.8918
Scale: 4
PSNR/SSIM: 26.48/0.8312
Scale: 8
PSNR/SSIM: 22.58/0.6877
Scale: 16
PSNR/SSIM: 19.28/0.5740



In [58]:
import statistics

scales = [2, 3, 4, 8, 16]
for scale in scales:
    print(f'Avg PSNR/SSIM {scale}x: {statistics.mean(psnr[scale]):.2f}/{statistics.mean(ssim[scale]):.4f}')

Avg PSNR/SSIM 2x: 33.71/0.9306
Avg PSNR/SSIM 3x: 30.92/0.8828
Avg PSNR/SSIM 4x: 28.47/0.8106
Avg PSNR/SSIM 8x: 24.47/0.6604
Avg PSNR/SSIM 16x: 21.53/0.5761


In [59]:
scales = [2, 3, 4, 8, 16]

def calc_result(dataset):
    print()
    print(dataset)
    psnr = {}
    ssim = {}
    do_test(psnr, ssim, f'/content/drive/Shareddrives/BTP Meets/datasets/test/{dataset}/')
    for scale in scales:
        print(f'Avg PSNR/SSIM {scale}x: {statistics.mean(psnr[scale]):.2f}/{statistics.mean(ssim[scale]):.4f}')

calc_result('Set14')
calc_result('BSDS100')
calc_result('Manga109')
calc_result('Urban100')


Set14
Avg PSNR/SSIM 2x: 30.25/0.8718
Avg PSNR/SSIM 3x: 27.90/0.7938
Avg PSNR/SSIM 4x: 25.99/0.7092
Avg PSNR/SSIM 8x: 23.11/0.5770
Avg PSNR/SSIM 16x: 20.80/0.5048

BSDS100
Avg PSNR/SSIM 2x: 30.98/0.8751
Avg PSNR/SSIM 3x: 27.66/0.7554
Avg PSNR/SSIM 4x: 26.71/0.6976
Avg PSNR/SSIM 8x: 24.19/0.5719
Avg PSNR/SSIM 16x: 22.27/0.5146

Manga109
Avg PSNR/SSIM 2x: 31.31/0.9409
Avg PSNR/SSIM 3x: 27.28/0.8648
Avg PSNR/SSIM 4x: 25.31/0.8012
Avg PSNR/SSIM 8x: 21.76/0.6626
Avg PSNR/SSIM 16x: 19.43/0.6008

Urban100
Avg PSNR/SSIM 2x: 26.90/0.8418
Avg PSNR/SSIM 3x: 25.23/0.7612
Avg PSNR/SSIM 4x: 23.17/0.6612
Avg PSNR/SSIM 8x: 20.84/0.5219
Avg PSNR/SSIM 16x: 19.10/0.4597
